## PDI - Rebalancing data for October 2022

## Setting

In [71]:
import pandas as pd
import numpy as np
import requests

In [72]:
import time

In [73]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [74]:
from defillama import DefiLlama
llama = DefiLlama()

In [75]:
import plotly.express as px
import plotly.graph_objects as go

In [76]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics** 
    1. The project must have a listing on Defi Llama
    1. The project’s token should have been listed on CoinGecko with pricing data at least 1 year prior to the date of inclusion in the index. 
    1. The project should have a token that is native to Ethereum L1 or L2. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project. 
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
    
    
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    1. The token must not have locking, minting or other patterns that would significantly disadvantage passive holders.


1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have in aggregate at least \$5mm of onchain liquidity across Uniswap v2, Uniswap v3, Sushiswap, Balancer v1, Balancer v2, Bancor v2 and Bancor v3.
    1. The price should experience no more than 4% price impact when executing a $100k trade.
    1. The token must have shown consistent DeFi liquidity on Ethereum.


1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [77]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### DefiLlama

In [78]:
protocols = llama.get_all_protocols()

In [79]:
top_defi_tokens = {} 
for pr in protocols[:1000]:
    if 'Ethereum' in pr['chains'] and pr['address'] and pr['address']!='-' and pr['symbol'] != '-':
        symbol = normalize_symb(pr['symbol']) 
        address = pr['address'].lower().split(":")[-1]
        top_defi_tokens[symbol] = {
            'symbol': symbol, 
            'address': address 
        }

#### CoinGecko

##### Tokens addresses

In [80]:
coins_list = pd.DataFrame(cg.get_coins_list(include_platform=True))

In [81]:
tokens = {} 
tokens_by_addr = {}
for index, coin in coins_list.iterrows():
    if len(coin['platforms']) >= 1 and 'ethereum' in list(coin['platforms'].keys()):
        symbol = normalize_symb(coin['symbol'])
        address = coin['platforms']['ethereum'].lower().split(":")[-1]
        token = {
            'symbol': symbol,
            'address': address 
        }
        tokens[symbol] = token
        tokens_by_addr[address] = token

In [82]:
tokens['USDC'], tokens['WETH']

({'symbol': 'USDC', 'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'},
 {'symbol': 'WETH', 'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'})

In [83]:
# These token's addresses cant be found in the coingecko list of eth addresses
exclude_symbols = []
for symb, token in top_defi_tokens.items():
    if token['address'] not in tokens_by_addr.keys():
        # print(token)
        exclude_symbols.append(token['symbol'])

In [84]:
for symb in exclude_symbols:
    del top_defi_tokens[symb]

##### Market data

In [85]:
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc')
for i in range(2, 20):
    time.sleep(10)
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)

In [86]:
tokens_market_data = cg_tokens_data.set_index(cg_tokens_data['symbol'].str.upper())

In [87]:
duplicated = tokens_market_data.index[tokens_market_data.index.duplicated(keep='first')]
tokens_market_data = tokens_market_data.drop(index=duplicated)

In [88]:
tokens_market_data = tokens_market_data.rename(index={"ETH": "WETH"})

### Token's supply check 

In [89]:
sp_passed_index = (tokens_market_data['circulating_supply'] / tokens_market_data['total_supply']) > 0.3
sp_passed = tokens_market_data[sp_passed_index]

### Liquidity check

In [90]:
api_urls = {
    'univ2': "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2",
    'univ3': "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3",
    'sushi': "https://api.thegraph.com/subgraphs/name/sushiswap/exchange",
    'bancorv2': "https://api-v2.bancor.network/tokens",
    'bancorv3': "https://api-v3.bancor.network/pools",
    'balancer': "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer",
    'balancerv2': "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-v2" 
} 

In [91]:
def univ2_get_liquidities(tokens_ids):
    query = """
        query pairs($tokens_ids: [ID!]!) {
            pairs (first:1000, 
                where: {
                    token0_in: $tokens_ids, 
                    token1_in: $tokens_ids,
                    reserveUSD_gt: 10000
                }){
                token0 {
                    id
                    symbol
                }
                token1 {
                    id
                    symbol
                }
                reserveUSD
            }
        }
    """
    url = api_urls['univ2']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    pairs = response.json()['data']['pairs']
    
    liqs = {}
    for pair in pairs:
        liq = float(pair['reserveUSD'])
        
        symb0 = normalize_symb(pair['token0']['symbol'])
        if symb0 in tokens_market_data.index and pair['token0']['id'].lower() in tokens_ids:
            liqs[symb0] = liqs.get(symb0, 0) + liq
        
        symb1 = normalize_symb(pair['token1']['symbol'])
        if symb1 in tokens_market_data.index and pair['token1']['id'].lower() in tokens_ids:
            liqs[symb1] = liqs.get(symb1, 0) + liq 
        
    return liqs
    

In [92]:
def univ3_get_liquidities(tokens_ids):
    query = """
        query pools($tokens_ids: [ID!]!) {
            pools (first:1000, 
                where: {
                    token0_in: $tokens_ids, 
                    token1_in: $tokens_ids,
                    totalValueLockedUSD_gt: 10000
                }){
                token0 {
                    id
                    symbol
                } 
                token1 {
                    id
                    symbol
                }
                totalValueLockedUSD
            }
        }
    """
    url = api_urls['univ3']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    pools = response.json()['data']['pools']
    
    liqs = {}
    for pool in pools:
        liq = float(pool['totalValueLockedUSD'])
        
        symb0 = normalize_symb(pool['token0']['symbol'])
        if symb0 in tokens_market_data.index and pool['token0']['id'].lower() in tokens_ids:
            liqs[symb0] = liqs.get(symb0, 0) + liq 
        
        symb1 = normalize_symb(pool['token1']['symbol'])
        if symb1 in tokens_market_data.index and pool['token1']['id'].lower() in tokens_ids:
            liqs[symb1] = liqs.get(symb1, 0) + liq 
            
    return liqs
    

In [93]:
def sushi_get_liquidities(tokens_ids):
    query = """
        query pairs($tokens_ids: [ID!]!) {
            pairs (first:1000, 
                where: {
                    token0_in: $tokens_ids, 
                    token1_in: $tokens_ids,
                    reserveUSD_gt: 10000
                }){
                token0 {
                    id
                    symbol
                }
                token1 {
                    id
                    symbol
                }
                reserveUSD
            }
        }
    """
    url = api_urls['sushi']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    pairs = response.json()['data']['pairs']
    
    liqs = {}
    for pair in pairs:
        liq = float(pair['reserveUSD'])
        
        symb0 = normalize_symb(pair['token0']['symbol'])
        if symb0 in tokens_market_data.index and pair['token0']['id'].lower() in tokens_ids:
            liqs[symb0] = liqs.get(symb0, 0) + liq
        
        symb1 = normalize_symb(pair['token1']['symbol'])
        if symb1 in tokens_market_data.index and pair['token1']['id'].lower() in tokens_ids:
            liqs[symb1] = liqs.get(symb1, 0) + liq
        
    return liqs
    

In [94]:
def bancorv2_get_liquidities(tokens_ids):
    url = api_urls['bancorv2'] 
    response = requests.get(url)
    tokens = response.json()['data']
    
    liqs = {}
    for token in tokens:
        if token['dlt_id'].lower() in tokens_ids:
            symb = normalize_symb(token['symbol'])
            liqs[symb] = float(token['liquidity']['usd'])
            
    return liqs
            

In [95]:
def bancorv3_get_liquidities(tokens_ids):
    url = api_urls['bancorv3'] 
    response = requests.get(url)
    pools = response.json()['data']
    
    liqs = {}
    for pool in pools:
        if pool['poolDltId'].lower() in tokens_ids:
            symb = normalize_symb(pool['name'])
            liqs[symb] = liqs.get(symb, 0) + float(pool['liquidity']['usd'])
            
    return liqs
            

In [96]:
def balancer_get_liquidities(tokens_ids):
    query = """
        query pools {
            pools (
                first: 1000,
                where: {
                  liquidity_gt: 10000
                }
            ){
                tokens {
                  address
                  symbol
                  balance
                }
                liquidity
          }
        }
    """
    url = api_urls['balancer']
    
    response = requests.post(url, json={'query': query})
    pools = response.json()['data']['pools']
    
    liqs = {}
    for pool in pools:
        liq = 0
        for token in pool['tokens']:
            symb = normalize_symb(token['symbol'])
            if symb in tokens_market_data.index: 
                liq += float(token['balance']) * tokens_market_data.loc[symb]['current_price']
            
        for token in pool['tokens']:
            symb = normalize_symb(token['symbol'])
            if symb not in tokens_market_data.index or token['address'].lower() not in tokens_ids:
                continue
                
            liqs[symb] = liqs.get(symb, 0) + liq
        
    return liqs
    

In [97]:
def balancerv2_get_liquidities(tokens_ids):
    query = """
        query pools {
            pools (
                first: 1000,
                where: {
                  totalLiquidity_gt: 10000
                }
            ){
                tokens {
                  address
                  symbol
                  balance
                }
                totalLiquidity
          }
        }
    """
    url = api_urls['balancerv2']
    
    response = requests.post(url, json={'query': query})
    pools = response.json()['data']['pools']
    
    liqs = {}
    for pool in pools:
        liq = float(pool['totalLiquidity'])
        
        for token in pool['tokens']:
            symb = normalize_symb(token['symbol'])
            if symb not in tokens_market_data.index or token['address'].lower() not in tokens_ids:
                continue
            
            liqs[symb] = liqs.get(symb, 0) + liq 
        
    return liqs
    

In [98]:
tokens['AMP']

{'symbol': 'AMP', 'address': '0xff20817765cb7f73d4bde2e66e067e58d11095c2'}

In [99]:
tokens_ids = [token['address'] for symb, token in top_defi_tokens.items()]

In [100]:
tokens_ids.append(tokens['WETH']['address'])
tokens_ids.append(tokens['USDC']['address'])

In [101]:
query_functions = {
    'univ2': univ2_get_liquidities,
    'univ3': univ3_get_liquidities, 
    'sushi': sushi_get_liquidities, 
    'bancorv2': bancorv2_get_liquidities,
    'bancorv3': bancorv3_get_liquidities,
    'balancer': balancer_get_liquidities, 
    'balancerv2': balancerv2_get_liquidities, 
}

In [102]:
liqs_dict = {}
for dex, func in query_functions.items():
    liqs_dict[dex] = func(tokens_ids)

In [103]:
liquidities = pd.DataFrame.from_dict(liqs_dict)

In [104]:
liquidities['sum'] = liquidities.sum(axis=1)
liquidities.sort_values('sum', ascending=False).head(50)

,univ2,univ3,sushi,bancorv2,bancorv3,balancer,balancerv2,sum
WETH,1.714270e+08,1.009666e+09,1.484776e+08,NaN,NaN,1.017373e+08,4.976651e+08,1.928973e+09
USDC,1.117003e+08,7.172938e+08,5.771782e+07,5.392411e+05,1.298686e+06,1.927322e+07,6.593874e+07,9.737618e+08
WBTC,1.044000e+07,3.403791e+08,1.781196e+07,3.216632e+06,3.214443e+06,4.037192e+06,2.623667e+07,4.053360e+08
BAL,2.416512e+04,5.182675e+04,NaN,0.000000e+00,0.000000e+00,3.481919e+06,1.532252e+08,1.567831e+08
AAVE,1.664068e+05,2.806977e+06,2.102707e+06,5.611888e+04,1.065293e+05,9.108236e+07,1.785231e+04,9.633895e+07
COMP,1.693658e+05,1.033010e+07,4.636372e+06,2.033344e+05,0.000000e+00,3.979393e+04,2.774985e+07,4.312881e+07
UNI,1.719536e+07,1.343729e+07,2.247226e+05,7.829000e+04,5.220076e+04,9.436989e+04,1.866986e+06,3.294922e+07
TEMPLE,NaN,4.300000e+04,NaN,NaN,NaN,NaN,3.244596e+07,3.248896e+07
LDO,1.742784e+05,1.193931e+07,6.718582e+06,NaN,NaN,NaN,5.866776e+06,2.469894e+07
BNT,7.559080e+04,NaN,NaN,2.350315e+07,NaN,NaN,NaN,2.357874e+07


In [105]:
liquidities.loc['AMP']

univ2          74727.268015
univ3         474368.480886
sushi         123357.052336
bancorv2       22926.806569
bancorv3       18315.461158
balancer                NaN
balancerv2              NaN
sum           713695.068964
Name: AMP, dtype: float64

In [106]:
tokens['AMP']

{'symbol': 'AMP', 'address': '0xff20817765cb7f73d4bde2e66e067e58d11095c2'}

In [107]:
liq_check = (liquidities['sum'] > 5e6).sort_values(ascending=False)
liq_check_passed = liq_check[liq_check]

In [108]:
lq_passed = sp_passed[
    sp_passed.index.isin(liq_check_passed.index)
]

In [109]:
# lq_passed

### Non quantative checks

In [110]:
exclude_list = [
    {
        'symbol': 'WBTC',
        'reason': 'Criteria 1.C'
    },
    {
        'symbol': 'WETH',
        'reason': 'Criteria 1.C'
    },
    {
        'symbol': 'ERN',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'NFTX',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'FXS',
        'reason': 'Criteria 1.G'
    },
    {
        'symbol': 'RGT',
        'reason': 'Deprecated asset'
    },
    {
        'symbol': 'SUPER',
        'reason': 'Not DeFi'
    },
    {
        'symbol': 'BOND',
        'reason': 'Criteria 1.I'
    },
    {
        'symbol': 'USDC',
        'reason': 'stable'
    },
    {
        'symbol': 'FEI',
        'reason': 'stable'
    },
    {
        'symbol': 'SYN',
        'reason': 'bridge'
    }
    
]
exclude_symbols = [token['symbol'] for token in exclude_list]

In [111]:
nq_passed = lq_passed[
    ~lq_passed.index.isin(exclude_symbols)
]

In [112]:
# nq_passed

### Retrieve historical data & livetime check

In [113]:
time.sleep(1)

exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for index, data in nq_passed[['id', 'symbol']].iterrows():
    id_ = data['id']
    symbol = data['symbol'].upper()
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', symbol])
    df_prices = df_prices[df_prices[symbol] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < 365:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < 365 days')
        exclude_list.append(symbol)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', symbol])
    df_mcaps = df_mcaps[df_mcaps[symbol] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < 365:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < 365 days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    time.sleep(10)
    
    

Excluding RBN, prices data available only for 355 < 365 days
Excluding TEMPLE, prices data available only for 263 < 365 days


In [114]:
lt_passed = nq_passed[
    ~nq_passed.index.isin(exclude_list)
]

In [115]:
lt_passed

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.730000,4308810700,19,5.716372e+09,90316325.0,5.930000,...,1.000000e+09,1.000000e+09,44.92,-87.27336,2021-05-03T05:25:04.822Z,1.030000,454.91354,2020-09-17T01:20:38.214Z,None,2022-09-26T10:14:05.204Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,74.640000,1053141486,51,1.194938e+09,96248725.0,77.010000,...,1.600000e+07,1.600000e+07,661.69,-88.71315,2021-05-18T21:19:59.514Z,26.020000,186.99049,2020-11-05T09:20:11.928Z,None,2022-09-26T10:13:46.233Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.590000,1030394714,52,1.596528e+09,45607908.0,1.720000,...,1.000000e+09,1.000000e+09,7.30,-78.16516,2021-08-20T08:35:20.158Z,0.406150,292.60646,2022-06-18T20:55:12.035Z,None,2022-09-26T10:14:04.941Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,692.110000,622061537,68,6.940233e+08,40180636.0,714.390000,...,9.776310e+05,1.005577e+06,6292.31,-89.04519,2021-05-03T21:54:29.333Z,168.360000,309.43268,2020-03-16T20:52:36.527Z,None,2022-09-26T10:14:02.443Z
SNX,havven,snx,Synthetix Network,https://assets.coingecko.com/coins/images/3406...,2.290000,546625965,78,6.841370e+08,43222546.0,2.450000,...,2.989486e+08,2.989486e+08,28.53,-91.97909,2021-02-14T01:12:38.505Z,0.034786,6478.86040,2019-01-06T00:00:00.000Z,"{'times': 3.571442108564588, 'currency': 'usd'...",2022-09-26T10:14:01.898Z
RPL,rocket-pool,rpl,Rocket Pool,https://assets.coingecko.com/coins/images/2090...,23.920000,448269691,91,4.482697e+08,1254534.0,24.250000,...,1.875905e+07,1.875905e+07,59.46,-59.80937,2021-11-09T04:03:31.003Z,0.008847,269993.69651,2018-08-28T00:00:00.000Z,None,2022-09-26T10:13:59.953Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,63.760000,438104582,92,6.390010e+08,60026352.0,64.080000,...,1.000000e+07,1.000000e+07,910.54,-92.98311,2021-05-12T02:29:08.794Z,26.410000,141.92077,2022-06-18T20:54:18.261Z,None,2022-09-26T10:14:00.738Z
1INCH,1inch,1inch,1inch,https://assets.coingecko.com/coins/images/1346...,0.610551,357474317,110,9.173994e+08,46897127.0,0.631201,...,1.500000e+09,1.500000e+09,8.65,-92.93070,2021-10-27T08:24:54.808Z,0.519281,17.78009,2022-06-18T20:10:15.819Z,None,2022-09-26T10:14:14.417Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,8165.190000,255976570,139,2.987838e+08,33455049.0,8390.450000,...,3.666600e+04,3.666600e+04,90787.00,-91.02426,2021-05-12T00:29:37.713Z,31.650000,25650.54998,2020-07-18T12:26:27.150Z,None,2022-09-26T10:14:34.124Z


In [116]:
liquidities.loc[lt_passed.index]

,univ2,univ3,sushi,bancorv2,bancorv3,balancer,balancerv2,sum
symbol,,,,,,,,
UNI,1.719536e+07,1.343729e+07,2.247226e+05,7.829000e+04,5.220076e+04,9.436989e+04,1.866986e+06,3.294922e+07
AAVE,1.664068e+05,2.806977e+06,2.102707e+06,5.611888e+04,1.065293e+05,9.108236e+07,1.785231e+04,9.633895e+07
LDO,1.742784e+05,1.193931e+07,6.718582e+06,NaN,NaN,NaN,5.866776e+06,2.469894e+07
MKR,1.435597e+06,4.609587e+06,3.150662e+05,2.079662e+05,9.610131e+04,3.806817e+05,NaN,7.044999e+06
SNX,1.032492e+06,3.443140e+06,6.235896e+05,7.481048e+05,1.057979e+05,1.742674e+05,1.314785e+06,7.442177e+06
RPL,NaN,9.573293e+06,NaN,2.524057e+04,5.459277e+04,NaN,1.452761e+06,1.110589e+07
COMP,1.693658e+05,1.033010e+07,4.636372e+06,2.033344e+05,0.000000e+00,3.979393e+04,2.774985e+07,4.312881e+07
1INCH,2.119773e+05,1.229324e+07,NaN,1.741319e+03,0.000000e+00,NaN,NaN,1.250696e+07
YFI,3.682164e+05,9.395375e+05,4.064278e+06,5.468041e+05,7.351824e+03,4.452638e+04,3.070897e+06,9.041611e+06


### Slippage check

To derive a slippage threshold, consider an abstract univ2-like pool with $5m\$$ overall liquidity. 
I.e if pool has reserves $R_1$, $R_2$ and $p_1$ and $p_2$ are the correspondent tokens prices (in USD), then
$$R_1p_1 = R_2p_2 = 2.5m \$$$

Swapping $x$ token1 for token2 will result in the output amount $y$ such that
$$R_1\cdot R_2 = (R_1 + x) \cdot (R_2 - y)$$
$$y = \frac{R_2\cdot x}{R_1 + x}$$
Then the delivery price will be
$$p_d = \frac{y}{x} = \frac{R_2}{R_1 + x}$$
The spot price may be estimated as a delivery price for relatively small input amount $x_0$
$$p_s = \frac{y_0}{x_0} = \frac{R_2}{R_1 + x_0}$$

Then the theoretical slippage for this pool will be
$$slippage = \frac{p_d}{p_s} - 1 = \frac{R_1+x_0}{R_1+x} - 1 = \frac{p_1R_1+p_1x_0}{p_1R_1+p_1x}-1$$

In [117]:
liq = 5*10**6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e3
trade_value = 1e5

In [118]:
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .5
max_slippage = round(2 * max_slippage, 2) / 2 
max_slippage

-0.04

In [119]:
url_0x = "https://api.0x.org/swap/v1/quote" 

In [120]:
decimals = 18
slippages = []
for symb, data in lt_passed.iterrows():
    query = {
        'buyToken': 'WETH',
        'sellToken': tokens[symb]['address'],
        'sellAmount': int(trade_value_tiny / data['current_price'] * 10 ** decimals),
    }
    
    # spot price is calculated as a price for 1000$ swap
    resp = requests.get(url_0x, params=query)
    swap = resp.json()
    spot_price = float(swap['price'])
    
    query['sellAmount'] = int(trade_value / data['current_price'] * 10 ** decimals)
    resp = requests.get(url_0x, params=query)
    swap = resp.json()
    del_price = float(swap['price'])
    
    slippage = del_price / spot_price - 1
    
    slippages.append({
        'symb': symb,
        'spot_price': spot_price,
        'delivery_price': del_price,
        'slippage': slippage
    })
    
    # time.sleep(1)

In [121]:
slippages_df = pd.DataFrame(slippages)
slippages_df

,symb,spot_price,delivery_price,slippage
0,UNI,4.344742e-03,4.325938e-03,-0.004328
1,AAVE,5.659437e-02,5.666949e-02,0.001327
2,LDO,1.209808e-03,1.204617e-03,-0.004291
3,MKR,5.260810e-01,5.170582e-01,-0.017151
4,SNX,1.746186e-03,1.711905e-03,-0.019632
5,RPL,1.821194e-02,1.798383e-02,-0.012526
6,COMP,4.832223e-02,4.809272e-02,-0.004750
7,1INCH,4.657594e-04,4.595883e-04,-0.013249
8,YFI,6.198716e+00,6.166567e+00,-0.005186
9,SUSHI,7.910990e-04,7.707142e-04,-0.025768


In [122]:
exclude_list = list(slippages_df[abs(slippages_df['slippage']) > abs(max_slippage)]['symb'])
exclude_list

['ICHI']

In [123]:
st_passed = lt_passed[
    ~lt_passed.index.isin(exclude_list)
]

### Marketcap ranking & top10 filtering

In [124]:
chosen_tokens = st_passed.sort_values("market_cap", ascending=False).iloc[:10]

In [125]:
chosen_tokens

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
symbol,,,,,,,,,,,,,,,,,,,,,
UNI,uniswap,uni,Uniswap,https://assets.coingecko.com/coins/images/1250...,5.730000,4308810700,19,5.716372e+09,90316325.0,5.930000,...,1.000000e+09,1.000000e+09,44.92,-87.27336,2021-05-03T05:25:04.822Z,1.030000,454.91354,2020-09-17T01:20:38.214Z,None,2022-09-26T10:14:05.204Z
AAVE,aave,aave,Aave,https://assets.coingecko.com/coins/images/1264...,74.640000,1053141486,51,1.194938e+09,96248725.0,77.010000,...,1.600000e+07,1.600000e+07,661.69,-88.71315,2021-05-18T21:19:59.514Z,26.020000,186.99049,2020-11-05T09:20:11.928Z,None,2022-09-26T10:13:46.233Z
LDO,lido-dao,ldo,Lido DAO,https://assets.coingecko.com/coins/images/1357...,1.590000,1030394714,52,1.596528e+09,45607908.0,1.720000,...,1.000000e+09,1.000000e+09,7.30,-78.16516,2021-08-20T08:35:20.158Z,0.406150,292.60646,2022-06-18T20:55:12.035Z,None,2022-09-26T10:14:04.941Z
MKR,maker,mkr,Maker,https://assets.coingecko.com/coins/images/1364...,692.110000,622061537,68,6.940233e+08,40180636.0,714.390000,...,9.776310e+05,1.005577e+06,6292.31,-89.04519,2021-05-03T21:54:29.333Z,168.360000,309.43268,2020-03-16T20:52:36.527Z,None,2022-09-26T10:14:02.443Z
SNX,havven,snx,Synthetix Network,https://assets.coingecko.com/coins/images/3406...,2.290000,546625965,78,6.841370e+08,43222546.0,2.450000,...,2.989486e+08,2.989486e+08,28.53,-91.97909,2021-02-14T01:12:38.505Z,0.034786,6478.86040,2019-01-06T00:00:00.000Z,"{'times': 3.571442108564588, 'currency': 'usd'...",2022-09-26T10:14:01.898Z
RPL,rocket-pool,rpl,Rocket Pool,https://assets.coingecko.com/coins/images/2090...,23.920000,448269691,91,4.482697e+08,1254534.0,24.250000,...,1.875905e+07,1.875905e+07,59.46,-59.80937,2021-11-09T04:03:31.003Z,0.008847,269993.69651,2018-08-28T00:00:00.000Z,None,2022-09-26T10:13:59.953Z
COMP,compound-governance-token,comp,Compound,https://assets.coingecko.com/coins/images/1077...,63.760000,438104582,92,6.390010e+08,60026352.0,64.080000,...,1.000000e+07,1.000000e+07,910.54,-92.98311,2021-05-12T02:29:08.794Z,26.410000,141.92077,2022-06-18T20:54:18.261Z,None,2022-09-26T10:14:00.738Z
1INCH,1inch,1inch,1inch,https://assets.coingecko.com/coins/images/1346...,0.610551,357474317,110,9.173994e+08,46897127.0,0.631201,...,1.500000e+09,1.500000e+09,8.65,-92.93070,2021-10-27T08:24:54.808Z,0.519281,17.78009,2022-06-18T20:10:15.819Z,None,2022-09-26T10:14:14.417Z
YFI,yearn-finance,yfi,yearn.finance,https://assets.coingecko.com/coins/images/1184...,8165.190000,255976570,139,2.987838e+08,33455049.0,8390.450000,...,3.666600e+04,3.666600e+04,90787.00,-91.02426,2021-05-12T00:29:37.713Z,31.650000,25650.54998,2020-07-18T12:26:27.150Z,None,2022-09-26T10:14:34.124Z


In [126]:
chosen_tokens_symbols = list(chosen_tokens.index)
prices_data = prices_data[chosen_tokens_symbols]
marketcaps = marketcaps[chosen_tokens_symbols]

In [127]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

In [128]:
# prices_data

In [129]:
# marketcaps

## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 30%.
* All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [130]:
weights = marketcaps.div(marketcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [131]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [132]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum(axis=1)
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum(axis=1), axis=0).mul(c, axis=0)
    
    return w

In [133]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum(axis=1), axis=0)
    
    return w

In [134]:
adjusted_weights = adjust_weights(weights, max_weight=0.3) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=0.005)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=0.3)

adjusted_weights = adjusted_weights.sort_values(adjusted_weights.last_valid_index(),ascending=False, axis = 1)
# adjusted_weights

In [135]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [136]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<76]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [137]:
last_weights = adjusted_weights.iloc[-1].dropna()

In [138]:
converted_last_weights = convert_weights(last_weights)

## Summary table

In [139]:
pdi = pd.DataFrame()
pdi.index = chosen_tokens.index
pdi['name'] = chosen_tokens['name']
pdi['market_cap'] = marketcaps.iloc[-1].astype(int)
pdi['price'] = prices_data.iloc[-1]
pdi['weight'] = last_weights
pdi['weight_converted'] = converted_last_weights
pdi['address'] = [top_defi_tokens[symb]['address'] for symb in pdi.index]

pdi = pdi[pdi['weight'] > 0]
pdi = pdi.sort_values("market_cap",ascending=False)
pdi

,name,market_cap,price,weight,weight_converted,address
symbol,,,,,,
UNI,Uniswap,4261519782,5.665178,0.300000,76,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
AAVE,Aave,1038167493,73.857866,0.147936,38,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
LDO,Lido DAO,1015411278,1.574242,0.144693,37,0x5a98fcbea516cf06857215779fd812ca3bef1b32
MKR,Maker,632463201,708.683694,0.090124,23,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
SNX,Synthetix Network,555412447,2.324715,0.079145,21,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f
RPL,Rocket Pool,443453217,23.582494,0.063191,17,0xd33526068d116ce69f19a9ee46f0bd304f21a51f
COMP,Compound,421932107,61.444237,0.060124,15,0xc00e94cb662c3520282e6f5717214004a7f26888
1INCH,1inch,350866736,0.602920,0.049998,12,0x111111111117dc0aa78b770fa6a738034120c302
YFI,yearn.finance,254351912,8144.360289,0.036244,9,0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e


In [140]:
# pdi[['address', 'weight_converted']].to_csv("reweight_info.csv", index=False)
print(pdi[['address', 'weight_converted']].to_csv(None, index=False))

address,weight_converted
0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,76
0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,38
0x5a98fcbea516cf06857215779fd812ca3bef1b32,37
0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,23
0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,21
0xd33526068d116ce69f19a9ee46f0bd304f21a51f,17
0xc00e94cb662c3520282e6f5717214004a7f26888,15
0x111111111117dc0aa78b770fa6a738034120c302,12
0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,9
0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,7

